In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
%fs head /FileStore/tables/json/department.csv

EmployeeID,DepartmentName,Client,OnboardedDate
1,Data Engineering,Funding Circle,2019-01-15
2,Data Engineering,Funding Circle,2018-11-19
3,Data Analytics,Funding Circle,2018-11-19
4,Data Analytics,Funding Circle,2018-12-16

In [0]:
dept_df=(spark
         .read
         .format("csv")
         .option("header","true")
         .option("inferSchema","true")
         .load("/FileStore/tables/json/department.csv"))

In [0]:
display(dept_df)

EmployeeID,DepartmentName,Client,OnboardedDate
1,Data Engineering,Funding Circle,2019-01-15
2,Data Engineering,Funding Circle,2018-11-19
3,Data Analytics,Funding Circle,2018-11-19
4,Data Analytics,Funding Circle,2018-12-16


In [0]:
emp_df=(spark
         .read
         .format("csv")
         .option("header","true")
         .option("inferSchema","true")
         .load("/FileStore/tables/json/employee.csv"))

In [0]:
display(emp_df)

ID,Name,HomeTown,Salary,JoiningDate
1,Arpit,Burhanpur,50000.0,2018-11-14
2,Benu,Bhubaneswar,100000.0,2018-11-19
3,Dilsher,Amritsar,100000.0,2018-11-19
4,Kiran,Bengaluru,50000.0,2018-11-15


In [0]:
dept_df.createOrReplaceTempView("department")

In [0]:
emp_df.createOrReplaceTempView("employee")

In [0]:
(spark.sql("SELECT * from department")).show()

+----------+----------------+--------------+-------------+
|EmployeeID|  DepartmentName|        Client|OnboardedDate|
+----------+----------------+--------------+-------------+
|         1|Data Engineering|Funding Circle|   2019-01-15|
|         2|Data Engineering|Funding Circle|   2018-11-19|
|         3|  Data Analytics|Funding Circle|   2018-11-19|
|         4|  Data Analytics|Funding Circle|   2018-12-16|
+----------+----------------+--------------+-------------+



In [0]:
display(spark.sql("SELECT * from employee"))

ID,Name,HomeTown,Salary,JoiningDate
1,Arpit,Burhanpur,50000.0,2018-11-14
2,Benu,Bhubaneswar,100000.0,2018-11-19
3,Dilsher,Amritsar,100000.0,2018-11-19
4,Kiran,Bengaluru,50000.0,2018-11-15


In [0]:
display(spark.sql("""
                  with employee_rank as (
                  select name,salary, dense_rank() over(order by salary desc) as Rank_Salary
                   from employee)
                   select * from employee_rank where Rank_Salary==2
                  """))

name,salary,Rank_Salary
Arpit,50000.0,2
Kiran,50000.0,2


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc
emp_rank_df=emp_df.withColumn("salaryRank", dense_rank().over(Window.orderBy(desc("Salary"))))
display(emp_rank_df.filter(emp_rank_df.salaryRank == 2))

ID,Name,HomeTown,Salary,JoiningDate,salaryRank
1,Arpit,Burhanpur,50000.0,2018-11-14,2
4,Kiran,Bengaluru,50000.0,2018-11-15,2
